# Feathr Fraud Detection Sample

This notebook illustrates the use of Feature Store to create a model that predicts the fraud status of transactions based on the user account data and trasaction data. All the data that was used in the notebook can be found here: https://github.com/microsoft/r-server-fraud-detection


In the following Notebook, we 
1. Install the latest Feathr code (to include some unreleased features) 
2. Define Environment Variables & `yaml_config` Settings 
3. Create `FeathrClient` and Define `FeatureAnchor`
4. `build_features` and `get_offline_features` 
5. Train Fraud Detection Model wih `KNeighborsClassifier`
6. `materialize_features` and `multi_get_online_features`
7. `register_features` and `list_registered_features`

## Setup Feathr Developer Environment

***Prior to running the notebook, if you have not deployed all the required resources, please refer to the guide here and follow the steps to do so: https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-arm.html***

In [4]:
# Install feathr from the latest codes in the repo. You may use `pip install feathr[notebook]` as well.
!pip install "git+https://github.com/feathr-ai/feathr.git#subdirectory=feathr_project&egg=feathr[notebook]"  

  Cloning https://github.com/feathr-ai/feathr.git to /tmp/pip-install-zsn6kl5o/feathr_36df02ae02da42a0a56397c6e9e058e7
  Running command git clone --filter=blob:none --quiet https://github.com/feathr-ai/feathr.git /tmp/pip-install-zsn6kl5o/feathr_36df02ae02da42a0a56397c6e9e058e7
  Resolved https://github.com/feathr-ai/feathr.git to commit 3ebaa49cf36dac90005fb2cbf5412d9103871e9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 7.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached confluent_kafka-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached azure_synapse_spark-0.7.0-py2.py3-none-any.whl (32 kB)
  Using

In [1]:
from datetime import datetime, timedelta
import glob
from math import sqrt
import os
import tempfile

from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from sklearn.model_selection import train_test_split

import feathr
from feathr import (
    FeathrClient,
    STRING, BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.utils.config import generate_config


print(f"Feathr version: {feathr.__version__}")

Feathr version: 0.9.0


In [2]:
RESOURCE_PREFIX = None  # TODO fill the value used to deploy the resources via ARM template
PROJECT_NAME = "fraud_detection"

# Currently support: 'azure_synapse', 'databricks', and 'local' 
SPARK_CLUSTER = "local"

# TODO fill values to use databricks cluster:
DATABRICKS_CLUSTER_ID = None             # Set Databricks cluster id to use an existing cluster
DATABRICKS_URL = None                    # Set Databricks workspace url to use databricks
DATABRICKS_WORKSPACE_TOKEN_VALUE = None  # Set Databricks workspace token to use databricks

# TODO fill values to use Azure Synapse cluster:
AZURE_SYNAPSE_SPARK_POOL = None  # Set Azure Synapse Spark pool name
AZURE_SYNAPSE_URL = None         # Set Azure Synapse workspace url to use Azure Synapse
ADLS_KEY = None                  # Set Azure Data Lake Storage key to use Azure Synapse

# An existing Feathr config file path. If None, we'll generate a new config based on the constants in this cell.
FEATHR_CONFIG_PATH = None

# If set True, use an interactive browser authentication to get the redis password.
USE_CLI_AUTH = False

In [6]:
# TODO remove this cell
RESOURCE_PREFIX = "juntest"
USE_CLI_AUTH = False

In [7]:
if SPARK_CLUSTER == "azure_synapse" and not os.environ.get("ADLS_KEY"):
    os.environ["ADLS_KEY"] = ADLS_KEY
elif SPARK_CLUSTER == "databricks" and not os.environ.get("DATABRICKS_WORKSPACE_TOKEN_VALUE"):
    os.environ["DATABRICKS_WORKSPACE_TOKEN_VALUE"] = DATABRICKS_WORKSPACE_TOKEN_VALUE

In [8]:
if USE_CLI_AUTH:
    !az login --use-device-code

## Permission
To run the cells below, you need additional permission: permission to your managed identity to access the keyvault, and permission to the user to access the Storage Blob. Run the following lines of command in the Cloud Shell in order to grant yourself the access.

```
userId=<email_id_of_account_requesting_access>
resource_prefix=<resource_prefix>
synapse_workspace_name="${resource_prefix}syws"
keyvault_name="${resource_prefix}kv"
objectId=$(az ad user show --id $userId --query id -o tsv)
az keyvault update --name $keyvault_name --enable-rbac-authorization false
az keyvault set-policy -n $keyvault_name --secret-permissions get list --object-id $objectId
az role assignment create --assignee $userId --role "Storage Blob Data Contributor"
az synapse role assignment create --workspace-name $synapse_workspace_name --role "Synapse Contributor" --assignee $userId
```

In [9]:
# Redis password
if 'REDIS_PASSWORD' not in os.environ:
    # Try to get all the required credentials from Azure Key Vault
    from azure.identity import AzureCliCredential, DefaultAzureCredential 
    from azure.keyvault.secrets import SecretClient

    # TODO assume the resources are deployed by using the ARM template. If not, please set your vault url name.
    vault_url = f"https://{RESOURCE_PREFIX}kv.vault.azure.net"
    if USE_CLI_AUTH:
        credential = AzureCliCredential(additionally_allowed_tenants=['*'],)
    else:
        credential = DefaultAzureCredential(
            exclude_interactive_browser_credential=False,
            additionally_allowed_tenants=['*'],
        )
    secret_client = SecretClient(vault_url=vault_url, credential=credential)
    retrieved_secret = secret_client.get_secret('FEATHR-ONLINE-STORE-CONN').value
    os.environ['REDIS_PASSWORD'] = retrieved_secret.split(",")[1].split("password=", 1)[1]

# feathr_output_path = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/feathr_output'

In [10]:
if FEATHR_CONFIG_PATH:
    config_path = FEATHR_CONFIG_PATH
else:
    config_path = generate_config(
        resource_prefix=RESOURCE_PREFIX,
        project_name=PROJECT_NAME,
        spark_config__spark_cluster=SPARK_CLUSTER,
        spark_config__azure_synapse__dev_url=AZURE_SYNAPSE_URL,
        spark_config__azure_synapse__pool_name=AZURE_SYNAPSE_SPARK_POOL,
        spark_config__databricks__workspace_instance_url=DATABRICKS_URL,
        databricks_cluster_id=DATABRICKS_CLUSTER_ID,
    )

with open(config_path, 'r') as f: 
    print(f.read())

api_version: 1
feature_registry:
  api_endpoint: https://juntestwebapp.azurewebsites.net/api/v1
offline_store:
  adls:
    adls_enabled: 'true'
  wasb:
    wasb_enabled: 'true'
online_store:
  redis:
    host: juntestredis.redis.cache.windows.net
    port: '6380'
    ssl_enabled: 'true'
project_config:
  project_name: fraud_detection
spark_config:
  spark_cluster: local
  spark_result_output_parts: '1'



## Initialize `Feathr Client`
- `FeathrClient`

In [11]:
client = FeathrClient(config_path=config_path, credential=credential)

2022-12-07 04:28:50.702 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - secrets__azure_key_vault__name not found in the config file.
2022-12-07 04:28:50.713 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__s3__s3_enabled not found in the config file.
2022-12-07 04:28:50.719 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__jdbc__jdbc_enabled not found in the config file.
2022-12-07 04:28:50.722 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__snowflake__snowflake_enabled not found in the config file.
2022-12-07 04:28:50.728 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - spark_config__local__feathr_runtime_location not found in the config file.
2022-12-07 04:28:50.730 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - spark_

## Define Features
- `HdfsSource`
- `TypedKey`
- `Feature`
- `FeatureAnchor`
- `DerivedFeature`

In [12]:
from feathr.datasets.constants import FRAUD_DETECTION_ACCOUNT_INFO_URL, FRAUD_DETECTION_TRANSACTIONS_URL

from feathr.datasets.utils import maybe_download

from pathlib import Path

In [13]:
# upload dataset if needed

account_info_file_path = str(Path(PROJECT_NAME, "account_info.csv"))
transactions_file_path = str(Path(PROJECT_NAME, "transactions.csv"))
maybe_download(
    src_url=FRAUD_DETECTION_ACCOUNT_INFO_URL,
    dst_filepath=account_info_file_path,
)
maybe_download(
    src_url=FRAUD_DETECTION_TRANSACTIONS_URL,
    dst_filepath=transactions_file_path,
)

100%|██████████| 13.0k/13.0k [00:00<00:00, 16.3kKB/s]
100%|██████████| 786/786 [00:00<00:00, 2.34kKB/s]


True

In [ ]:
# Upload files to cloud
if client.spark_runtime == "local" or (client.spark_runtime == "databricks" and is_databricks()):
    # In local mode, we can use the same data path as the source.
    # If the notebook is running on databricks, DATA_FILE_PATH should be already a dbfs path.
    data_source_path = DATA_FILE_PATH
else:
    # Otherwise, upload the local file to the cloud storage (either dbfs or adls).
    data_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(DATA_FILE_PATH)    

### Account Features

In [ ]:
#Refer to <https://feathr.readthedocs.io/en/latest/feathr.html> to learn more about the details of each method
account_info = HdfsSource(
    name="AccountData",
    path="wasbs://frauddata@feathrdatastorage.blob.core.windows.net/account_out_small.csv",
    event_timestamp_column="transactionDate",
    timestamp_format="yyyyMMdd",
)

accountId = TypedKey(key_column="accountID",
                       key_column_type=ValueType.INT32,
                       description="account id")

account_country = Feature(name="account_country",
                           key=accountId,
                           feature_type=STRING, 
                           transform="accountCountry")

is_user_registered = Feature(name="is_user_registered",
                                    key=accountId,
                                    feature_type=BOOLEAN,
                                    transform="isUserRegistered==TRUE")

num_payment_rejects_1d_per_user = Feature(name="num_payment_rejects_1d_per_user",
                                    key=accountId,
                                    feature_type=INT32,
                                    transform="numPaymentRejects1dPerUser")

account_age = Feature(name="account_age",
                      key=accountId,
                      feature_type=INT32,
                      transform="accountAge")
                                    
features = [
    account_country,
    account_age,
    is_user_registered,
    num_payment_rejects_1d_per_user
]

account_anchor = FeatureAnchor(name="account_features",
                               source=account_info,
                               features=features)

### Transaction Features

In [ ]:
# # #Refer to <https://feathr.readthedocs.io/en/latest/feathr.html> to learn more about the details of each method

transaction_data = HdfsSource(name="transaction_data",
                          path="wasbs://frauddata@feathrdatastorage.blob.core.windows.net/transaction_out_small.csv",
                          event_timestamp_column="transactionDate",
                          timestamp_format="yyyyMMdd")

transaction_id = Feature(name="transaction_id",
                        key=accountId,
                        feature_type=STRING,
                        transform="transactionID")

transaction_currency_code = Feature(name="transaction_currency_code",
                                    key=accountId,
                                    feature_type=STRING,
                                    transform="transactionCurrencyCode")
                           
transaction_amount = Feature(name="transaction_amount",
                            key=accountId,
                            feature_type=FLOAT,
                            transform="transactionAmount")

transaction_device_id = Feature(name="transaction_device_id",
                                key=accountId,
                                feature_type=FLOAT,
                                transform="transactionDeviceId")

transaction_ip_address = Feature(name="transaction_ip_address",
                                key=accountId,
                                feature_type=FLOAT,
                                transform="transactionIPaddress")

transaction_time = Feature(name="transaction_time",
                            key=accountId,
                            feature_type=INT32,
                            transform="transactionTime")

fraud_status = Feature(name="fraud_status",
                       key=accountId,
                       feature_type=STRING,
                       transform="fraud_tag")

features = [
    transaction_id,
    transaction_amount,
    transaction_device_id,
    transaction_ip_address,
    transaction_time,
    transaction_currency_code,
    fraud_status
]

transaction_feature_anchor = FeatureAnchor(name="transaction_features",
                                            source=transaction_data,
                                            features=features)

### Transaction Aggregated Features

In [ ]:
# average amount of transaction in the past week
transactions_aggr = HdfsSource(name="transactions_aggr",
                          path="wasbs://frauddata@feathrdatastorage.blob.core.windows.net/transaction_out_small.csv",
                          event_timestamp_column="transactionDate",
                          timestamp_format="yyyyMMdd")

# average amount of transaction in that week
avg_transaction_amount = Feature(name="avg_transaction_amount",
                                key=accountId,
                                feature_type=FLOAT,
                                transform=WindowAggTransformation(agg_expr="cast_float(transactionAmount)",
                                                            agg_func="AVG",
                                                            window="7d"))

# number of transaction that took place in a day
num_trasaction_count_in_day = Feature(name="num_trasaction_count_in_day",
                                    key=accountId,
                                    feature_type=INT32,
                                    transform=WindowAggTransformation(agg_expr="transactionID",
                                                                agg_func="COUNT",
                                                                window="1d"))

# Amount of transaction that took place in a day
total_transaction_amount_in_day = Feature(name="total_transaction_amount_in_day",
                                    key=accountId,
                                    feature_type=FLOAT,
                                    transform=WindowAggTransformation(agg_expr="cast_float(transactionAmount)",
                                                                agg_func="SUM",
                                                                window="1d"))

# average time of transaction in the past week
avg_transaction_time = Feature(name="avg_transaction_time",
                            key=accountId,
                            feature_type=INT32,
                            transform=WindowAggTransformation(agg_expr="cast_float(transactionTime)",
                                                          agg_func="AVG",
                                                          window="7d"))                                                            

# total number of currency used for transaction in the past week
num_currency_type_in_week = Feature(name="num_currency_type_in_week",
                                    key=accountId,
                                    feature_type=INT32,
                                    transform=WindowAggTransformation(agg_expr="transactionCurrencyCode",
                                                                    agg_func="COUNT",
                                                                    window="7d"))

# number of different ip address used for transaction in the past week
num_ip_address_count = Feature(name="num_ip_address_count",
                        key=accountId,
                        feature_type=INT32,
                        transform=WindowAggTransformation(agg_expr="transactionIPaddress",
                                                          agg_func="COUNT",
                                                          window="7d"))

# number of devices used for the transaction in the past week
num_device_count = Feature(name="num_device_count",
                            key=accountId,
                            feature_type=INT32,
                            transform=WindowAggTransformation(agg_expr="transactionDeviceId",
                                                            agg_func="COUNT",
                                                            window="7d"))

# find the time of most recent transaction
time_most_recent_transaction = Feature(name="time_most_recent_transaction",
                                        key=accountId,
                                        feature_type=INT32,
                                        transform=WindowAggTransformation(agg_expr="transactionTime",
                                                                        agg_func="LATEST",
                                                                        window="7d"))

features = [
    avg_transaction_amount,
    avg_transaction_time,
    total_transaction_amount_in_day,
    num_trasaction_count_in_day,
    num_currency_type_in_week,
    num_ip_address_count,
    num_device_count,
    time_most_recent_transaction
]

aggr_anchor = FeatureAnchor(name="transaction_aggr_features",
                            source=transactions_aggr,
                            features=features)

### Derived Features
- `DerivedFeature`

In [ ]:
# derived features
feature_diff_current_and_avg_amount = DerivedFeature(name="feature_diff_current_and_avg_amount",
                                                    key=accountId,
                                                    feature_type=FLOAT,
                                                    input_features=[
                                                        transaction_amount, avg_transaction_amount],
                                                    transform="transaction_amount - avg_transaction_amount")

feature_time_pass_after_most_recent_transaction = DerivedFeature(name="feature_time_pass_after_most_recent_transaction",
                                                                key=accountId,
                                                                feature_type=INT32,
                                                                input_features=[
                                                                    transaction_time, time_most_recent_transaction],
                                                                transform="cast_int(transaction_time) - cast_int(time_most_recent_transaction)")

## Build Defined Features
- `build_features`

In [ ]:
client.build_features(anchor_list=[account_anchor, transaction_feature_anchor, aggr_anchor], 
                      derived_feature_list=[feature_time_pass_after_most_recent_transaction, feature_diff_current_and_avg_amount])

## Get Offline Features
- `FeatureQuery`
- `ObservationSettings`
- `get_offline_features`
- `feathr_spark_launcher.download_result`

In [ ]:
if client.spark_runtime == 'databricks':
    output_path = 'dbfs:/feathrfrauddetection_test.avro'
else:
    output_path = feathr_output_path

feature_query = FeatureQuery(
    feature_list=["account_country",
                  "transaction_time",
                  "num_currency_type_in_week",
                  "num_trasaction_count_in_day",
                  "total_transaction_amount_in_day",
                  "fraud_status",
                  "is_user_registered",
                  "avg_transaction_amount",
                  "num_ip_address_count",
                  "num_device_count",
                  "time_most_recent_transaction",
                  "feature_diff_current_and_avg_amount",
                  "feature_time_pass_after_most_recent_transaction"], key=accountId)
                    
settings = ObservationSettings(
    observation_path="wasbs://frauddata@feathrdatastorage.blob.core.windows.net/observation_out_small.csv",
    event_timestamp_column="transactionDate",
    timestamp_format="yyyyMMdd")
    
client.get_offline_features(observation_settings=settings,
                            feature_query=feature_query,
                            output_path=output_path)
client.wait_job_to_finish(timeout_sec=10000000000)

In [ ]:
import pandas as pd
import pandavro as pdx
import glob
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from feathr import BackfillTime, MaterializationSettings, RedisSink

In [ ]:
def get_result_df(client: FeathrClient) -> pd.DataFrame:
    """Download the job result dataset from cloud as a Pandas dataframe."""
    res_url = client.get_job_result_uri(block=True, timeout_sec=600)
    tmp_dir = tempfile.TemporaryDirectory()
    client.feathr_spark_launcher.download_result(result_path=res_url, local_folder=tmp_dir.name)
    dataframe_list = []
    # assuming the result are in avro format
    for file in glob.glob(os.path.join(tmp_dir.name, '*.avro')):
        dataframe_list.append(pdx.read_avro(file))
    vertical_concat_df = pd.concat(dataframe_list, axis=0)
    tmp_dir.cleanup()
    return vertical_concat_df

df_res = get_result_df(client)

## Feature Visualization

In [ ]:
filepath = Path('./result_out.csv')
df_res.to_csv(filepath, index=False) 
df_res.reset_index()
df_res

## Train Fraud Detection Model with Calculated Features

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
import seaborn as sns

final_df = df_res
final_df.drop(['accountID'], axis=1, inplace=True, errors='ignore')
final_df.drop(['transactionDate'], axis=1, inplace=True, errors='ignore')
final_df.drop(['account_country'], axis=1, inplace=True, errors='ignore')
final_df = final_df.fillna(0)

x_train, x_test, y_train, y_test = train_test_split(final_df.drop(["fraud_status"], axis=1),
                                                    final_df["fraud_status"],
                                                    test_size=0.20,
                                                    random_state=0)
  
K = []
training = []
test = []
scores = {}
  
for k in range(2, 21):
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(x_train, y_train)
  
    training_score = clf.score(x_train, y_train)
    test_score = clf.score(x_test, y_test)
    K.append(k)
  
    training.append(training_score)
    test.append(test_score)
    scores[k] = [training_score, test_score]

for keys, values in scores.items():
    print(keys, ':', values)

## Materialize Features in Redis
- `BackfillTime`
- `RedisSink`
- `materialize_features`
- `multi_get_online_features`

In [ ]:
backfill_time = BackfillTime(start=datetime(
    2013, 4, 7), end=datetime(2013, 4, 7), step=timedelta(days=1))
redisSink = RedisSink(table_name="fraudDetectionDemoFeature")
settings = MaterializationSettings("fraudDetectionDemoFeature",
                                   backfill_time=backfill_time,
                                   sinks=[redisSink],
                                   feature_names=["fraud_status"])

client.materialize_features(settings, allow_materialize_non_agg_feature =True)
client.wait_job_to_finish(timeout_sec=5000)

In [ ]:
client.multi_get_online_features('fraudDetectionDemoFeature', ['1759222192247110', '914800996051170'], [
                                 "fraud_status"])

In [ ]:
client.multi_get_online_features('fraudDetectionDemoFeature', ['1759222192247110', '914800996051170', '844428033864668'], [
                                 "fraud_status"])

## Register Features with Registry APIs
- `register_features`
- `list_registered_features`
- Above queries are send to a Standard Registry API Service (both `Purview` and `SQL` backend are supported)
- More friendly interface with detailed lineage can be found in: [Feathr UI](https://feathr-sql-registry.azurewebsites.net/)

In [ ]:
client.register_features()
client.list_registered_features(project_name="fraud_detection_test")